# 10DOF IMU Example

This example shows how to use the [10DOF IMU sensor](http://www.dfrobot.com.cn/goods-1860.html) on the board. This IMU has the ability to acquire data from accelerometer, gyroscope and magnetometer, process the data in embedded MCU.

For this notebook, a PYNQ Arduino is required, and the IMU sensor should connect to the I2C interface.

### 1. Load overlay

In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("Robot.bit")

### 2. Read single axis data
This example shows on how to get a Axis data from the 10DOF IMU sensor.

10DOF IMU sensor contains accelerometer, gyroscope and magnetometer, which can provides real-time raw sample of the sensed acceleration, angular rate and magnetic field.

In [ ]:
import math
from pynq.lib.arduino import TenDOF_IMU
IMU = TenDOF_IMU(base.ARDUINO, channel = 1)

In [ ]:
[ax, ay, az, gx, gy, gz, mx, my, mz] = IMU.get_axis()
print("Acceleration is",float("{0:.2f}".format(ax)),float("{0:.2f}".format(ay)),float("{0:.2f}".format(az)))
print("Angular rate is",float("{0:.2f}".format(gx)),float("{0:.2f}".format(gy)),float("{0:.2f}".format(gz)))
print("Magnetic field is",float("{0:.2f}".format(mx)),float("{0:.2f}".format(my)),float("{0:.2f}".format(mz)))

### 3. Read temperature and pressure data
This example shows on how to get a temperature and pressure data from the 10DOF IMU sensor.

It's worth noting that the pressure retrieved from the 10DOF IMU sensor is an integer. And its unit is Pascal.

In [ ]:
temp = IMU.get_temperature()
pres = IMU.get_pressure()
print("Temperature is",float("{0:.2f}".format(temp)))
print("Pressure is",pres)

### 4. Advanced: Read Euler angle and Quaternion
The 10DOF IMU sensor possess a on-chip MCU which can calculate the Euler angle and Quaternion based on the information from the three sensors which may relieve the user's work on sensor fusion.This example shows how to get the data from the sensor.

In [ ]:
[head, roll, pitch] = IMU.get_eul()
print("The Euler angle-- Head:",float("{0:.2f}".format(head)),"Roll:",float("{0:.2f}".format(roll)),"Pitch:",float("{0:.2f}".format(pitch)))

In [ ]:
[w, x, y, z] = IMU.get_qua()
print("The Quaternion-- w:",w,"x:",x,"y:",y,"z:",z)

Although we can obtain the Euler angle and Quaternion, but we actually don't know what the orientation of the car is. You may imagine where the car point to by using the Euler angle. But with the Quaternion? No way!

Therefore, the demo below can visualize the IMU's orientation at reat time.

Firstly, initialize the [matplotlib](https://matplotlib.org/index.html) before visualization.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
#from IPython import display
import numpy as np

ax = plt.axes(projection='3d')
plt.cla();

A **qua2vec** function can help us "translate" the quaternion.It receive a vector in the sensor's reference frame, rotate it to the Earth's reference frame and output.

And we will go into a forever loop to update the animation.

In [ ]:
from IPython import display
plt.ion()
fig = plt.figure(1)
ax = fig.gca(projection='3d')
ax.set_zlim3d(-1, 1)
ax.set_ylim3d(-1, 1)
ax.set_xlim3d(-1, 1)
ax.set_aspect("equal")

def qua2vec(qua, a_x, a_y, a_z):
    a_xe = 2*a_x * (0.5 - qua[2] * qua[2] - qua[3] * qua[3]) + 2*a_y * (qua[1]*qua[2] - qua[0]*qua[3]) + 2*a_z * (qua[1]*qua[3] + qua[0]*qua[2]);
    a_ye = 2*a_x * (qua[1]*qua[2] + qua[0]*qua[3]) + 2*a_y * (0.5 - qua[1] * qua[1] - qua[3] * qua[3]) + 2*a_z * (qua[2]*qua[3] - qua[0]*qua[1]);
    a_ze = 2*a_x * (qua[1]*qua[3] - qua[0]*qua[2]) + 2*a_y * (qua[2]*qua[3] + qua[0]*qua[1]) + 2*a_z * (0.5 - qua[1] * qua[1] - qua[2] * qua[2]);
    return [a_xe, a_ye, a_ze]
while True:
    qua = IMU.get_qua()
    [a_xe, a_ye, a_ze] = qua2vec(qua, 1, 0, 0);
    q0 = ax.quiver(0, 0, 0, a_xe, a_ye, a_ze, color='red', length=1.0)
    [a_xe, a_ye, a_ze] = qua2vec(qua, 0, 1, 0);
    q1 = ax.quiver(0, 0, 0, a_xe, a_ye, a_ze, color='green', length=1.0)
    [a_xe, a_ye, a_ze] = qua2vec(qua, 0, 0, 1);
    q2 = ax.quiver(0, 0, 0, a_xe, a_ye, a_ze, color='blue', length=1.0)
    display.clear_output(wait=True)
    display.display(fig)
    q0.remove()
    q1.remove()
    q2.remove()